In [9]:
import pandas as pd
import numpy as np

In [10]:
def rebalance_indexes(
        index_indexes: list[str],
        index_input_df: pd.DataFrame, 
        index_asset_cap: int, 
        index_total_cap: int
):
    output_df = pd.DataFrame(
        columns=["Amount", "Value", "Percentage"],
        index=index_indexes
    )
    total_market_cap = index_input_df.MarketCap.sum()
    
    def share_of_market():
        indexes_dict = {}.fromkeys(index_indexes)
        for key, value in indexes_dict.items():
            indexes_dict[key] = index_input_df.MarketCap[key] / total_market_cap
        output_df.Percentage = indexes_dict.values()
    
    def rebalance_and_fill():
        nonlocal index_asset_cap
        if float(index_asset_cap) < 1 / (len(index_indexes)):
            output_df.loc[:, 'Percentage'] = 101  # force all values above asset cap
            index_asset_cap = 1 / (len(index_indexes))
        share_remaining = 1.00
        market_cap_remaining = total_market_cap
        for index, row in output_df.iterrows():
            share: float
            if row['Percentage'] > index_asset_cap:
                share = index_asset_cap
            else:
                share = share_remaining * index_input_df.MarketCap[index] / market_cap_remaining
            value = share * index_total_cap
            output_df.loc[index, 'Percentage'] = share
            output_df.loc[index, 'Value'] = value
            output_df.loc[index, 'Amount'] = value / index_input_df.Price[index]
            share_remaining -= share
            market_cap_remaining -= index_input_df.MarketCap[index]
    share_of_market()
    rebalance_and_fill()
    return output_df

# Sanity check

In [11]:
asset_cap = 1
total_cap = 10000

indexes = ['BTC', 'ETH', 'LTC']

input_dict = {
    "MarketCap": [
        20000, 10000, 5000
    ],
    "Price": [
        50, 25, 10
    ]
}
input_df = pd.DataFrame(
    data=input_dict,
    dtype=np.float64,
    index=indexes
)

print(f"Index Data: \n {input_df}\n")
print(f"Asset capitalisation: {asset_cap}")
print(f"Total capitalisation: {total_cap}\n")

results = rebalance_indexes(
    index_indexes=indexes,
    index_input_df=input_df,
    index_asset_cap=asset_cap,
    index_total_cap=total_cap
)
print("Balanced indexes:")
print(results)

Index Data: 
      MarketCap  Price
BTC    20000.0   50.0
ETH    10000.0   25.0
LTC     5000.0   10.0

Asset capitalisation: 1
Total capitalisation: 10000

Balanced indexes:
         Amount        Value  Percentage
BTC  114.285714  5714.285714    0.571429
ETH  114.285714  2857.142857    0.285714
LTC  142.857143  1428.571429    0.142857


## Test results

In [12]:
calculated_total_cap = round(results.Value.sum())
assert calculated_total_cap == total_cap, \
    "Total cap does not not match expected value:\n" \
    f"\t expected: {total_cap}, got {calculated_total_cap}"
assert results.Percentage.max() <= asset_cap

# Example 1

In [13]:
asset_cap = 0.5
total_cap = 10000

indexes = ['BTC', 'ETH', 'LTC']

input_dict = {
    "MarketCap": [
        20000, 10000, 5000
    ],
    "Price": [
        50, 25, 10
    ]
}
input_df = pd.DataFrame(
    data=input_dict,
    dtype=np.float64,
    index=indexes
)

print(f"Index Data: \n {input_df}\n")
print(f"Asset capitalisation: {asset_cap}")
print(f"Total capitalisation: {total_cap}\n")

results = rebalance_indexes(
    index_indexes=indexes,
    index_input_df=input_df,
    index_asset_cap=asset_cap,
    index_total_cap=total_cap
)
print("Balanced indexes:")
print(results)

Index Data: 
      MarketCap  Price
BTC    20000.0   50.0
ETH    10000.0   25.0
LTC     5000.0   10.0

Asset capitalisation: 0.5
Total capitalisation: 10000

Balanced indexes:
         Amount        Value  Percentage
BTC       100.0       5000.0    0.500000
ETH  133.333333  3333.333333    0.333333
LTC  166.666667  1666.666667    0.166667


## Test results

In [14]:
calculated_total_cap = round(results.Value.sum())
assert calculated_total_cap == total_cap, \
    "Total cap does not not match expected value:\n" \
    f"\t expected: {total_cap}, got {calculated_total_cap}"
assert results.Percentage.max() <= asset_cap

# Example 2

In [15]:
asset_cap = 0.1
total_cap = 10000

indexes = ['BTC', 'ETH', 'LTC']

input_dict = {
    "MarketCap": [
        20000, 10000, 5000
    ],
    "Price": [
        50, 25, 10
    ]
}
input_df = pd.DataFrame(
    data=input_dict,
    dtype=np.float64,
    index=indexes
)

print(f"Index Data: \n {input_df}\n")
print(f"Asset capitalisation: {asset_cap}")
print(f"Total capitalisation: {total_cap}\n")

results = rebalance_indexes(
    index_indexes=indexes,
    index_input_df=input_df,
    index_asset_cap=asset_cap,
    index_total_cap=total_cap
)
print("Balanced indexes:")
print(results)

Index Data: 
      MarketCap  Price
BTC    20000.0   50.0
ETH    10000.0   25.0
LTC     5000.0   10.0

Asset capitalisation: 0.1
Total capitalisation: 10000

Balanced indexes:
         Amount        Value  Percentage
BTC   66.666667  3333.333333    0.333333
ETH  133.333333  3333.333333    0.333333
LTC  333.333333  3333.333333    0.333333


## Test results

In [16]:
calculated_total_cap = round(results.Value.sum())
assert calculated_total_cap == total_cap, \
    "Total cap does not not match expected value:\n" \
    f"\t expected: {total_cap}, got {calculated_total_cap}"
assert results.Percentage.max() <= max(asset_cap, 1 / (len(indexes))), \
    f"Asset cap exceeded: max percentage is {results.Percentage.max()}"